### Data sets were acquired from https://www.kaggle.com/c/titanic

In [1]:
# Import libraries and the data set
import pandas as pd
import numpy as np
import sidetable
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('train.csv', index_col=['PassengerId'])
X_test = pd.read_csv('test.csv', index_col=['PassengerId'])

In [2]:
# Have a look at the first five rows
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Column __Name__ and __Ticket__ has nothing to do with predicting __Survived__, it will be dropped

In [3]:
# Column 'Name' and 'Ticket' have nothing to do with predicting 'Survived', they will be dropped
df.drop(columns=['Name'], inplace=True)
X_test.drop(columns=['Name'], inplace=True)

In [4]:
# See the missing data in columns
df.stb.missing(style=True)

,missing,total,percent
Cabin,687,891,77.10%
Age,177,891,19.87%
Embarked,2,891,0.22%
Survived,0,891,0.00%
Pclass,0,891,0.00%
Sex,0,891,0.00%
SibSp,0,891,0.00%
Parch,0,891,0.00%
Ticket,0,891,0.00%
Fare,0,891,0.00%


As column __Cabin__ has 77% missing values, the column will be dropped.
<br>Column __Age__ can be imputed.
<br>As column __Embarked__ has 2 missing rows, the rows will be dropped.

In [5]:
# As Pclass includes fare, I will drop 'Fare'
df.drop(columns=['Ticket', 'Cabin'], inplace=True, axis=1)

In [6]:
X_test.stb.missing(style=True)

,missing,total,percent
Cabin,327,418,78.23%
Age,86,418,20.57%
Fare,1,418,0.24%
Pclass,0,418,0.00%
Sex,0,418,0.00%
SibSp,0,418,0.00%
Parch,0,418,0.00%
Ticket,0,418,0.00%
Embarked,0,418,0.00%


As column __Cabin__ has 77% missing values, the column will be dropped.
<br>Column __Age__ can be imputed.
<br>As column __Fare__ has 2 missing rows, the rows will be dropped.

In [7]:
# As column Cabin is missing a lot, and column Ticket has a lot of unique values, which is hard to encode, I will drop them
X_test.drop(columns=['Ticket', 'Cabin'], axis=1, inplace=True)

In [8]:
# Recheck the missing data in columns
df.stb.missing(style=True)

,missing,total,percent
Age,177,891,19.87%
Embarked,2,891,0.22%
Survived,0,891,0.00%
Pclass,0,891,0.00%
Sex,0,891,0.00%
SibSp,0,891,0.00%
Parch,0,891,0.00%
Fare,0,891,0.00%


In [9]:
X_test.stb.missing(style=True)

,missing,total,percent
Age,86,418,20.57%
Fare,1,418,0.24%
Pclass,0,418,0.00%
Sex,0,418,0.00%
SibSp,0,418,0.00%
Parch,0,418,0.00%
Embarked,0,418,0.00%


In [10]:
# Get X_train and y_train
y_train = df.Survived.copy()
X_train = df.drop(columns='Survived').copy()

In [11]:
# Retrive num_cols and cat_cols
num_cols = list(X_train._get_numeric_data().columns)
cat_cols = list(set(X_train.columns) - set(num_cols))

In [12]:
# Encode cat_cols
X_train = pd.get_dummies(data=X_train, columns=cat_cols, drop_first=True)
X_test = pd.get_dummies(data=X_test, columns=cat_cols, drop_first=True)

### As no categorical columns have missing data, there is no need to impute them

In [13]:
# Impute numeric data
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
num_imputer = IterativeImputer()
num_imputer.fit(X_train[num_cols])
X_train[num_cols] = num_imputer.transform(X_train[num_cols])
X_test[num_cols] = num_imputer.transform(X_test[num_cols])

In [14]:
def scale(X, num_cols):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X[num_cols])
    X_scaled = pd.DataFrame(data=X_scaled, columns=num_cols, index=X.index)
    return X_scaled

In [15]:
# Scaling data
X_scaled = scale(X_train, num_cols)
X_test_scaled = scale(X_test, num_cols)

In [16]:
# Update cat_cols
cat_cols = set(X_train.columns) - set(num_cols)

In [17]:
X_train = pd.concat([X_scaled, X_train[cat_cols]], axis=1)

In [18]:
# Upsample y_train
from imblearn.combine import SMOTEENN
sampler = SMOTEENN()
X_train, y_train = sampler.fit_resample(X_train, y_train)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train)

In [20]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Activation, Dense, Conv2D, Flatten, MaxPool2D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

# Create the keras tuner model.
def build_model(hp):
    hp_drop_out = hp.Float('dropout', 0, 0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model = Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32), activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(hp_drop_out))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                loss='binary_crossentropy',
                metrics=['accuracy'])
    return model

In [21]:
import kerastuner as kt

tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=16,
                     overwrite=False)

In [22]:
# checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy',verbose=0, save_best_only=True)
es = EarlyStopping(monitor='val_accuracy', verbose=0, patience=10, min_delta=1e-3)
callbacks_list = [es]

In [23]:
tuner.search(X_train, y_train, epochs=64, batch_size=32, validation_data=(X_val, y_val), callbacks=callbacks_list)

Trial 254 Complete [00h 00m 13s]
val_accuracy: 0.9402984976768494

Best val_accuracy So Far: 0.9925373196601868
Total elapsed time: 00h 36m 56s
INFO:tensorflow:Oracle triggered exit


In [24]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [26]:
# Build the model with the optimal hyperparameters and train it on the data for 64 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=64, batch_size=32, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1

Epoch 1/64
14/14 [==============================] - 3s 30ms/step - loss: 0.4875 - accuracy: 0.7792 - val_loss: 0.6598 - val_accuracy: 0.6542
Epoch 2/64
14/14 [==============================] - 0s 9ms/step - loss: 0.1240 - accuracy: 0.9554 - val_loss: 0.6300 - val_accuracy: 0.7944
Epoch 3/64
14/14 [==============================] - 0s 9ms/step - loss: 0.1160 - accuracy: 0.9712 - val_loss: 0.6181 - val_accuracy: 0.6636
Epoch 4/64
14/14 [==============================] - 0s 9ms/step - loss: 0.0993 - accuracy: 0.9678 - val_loss: 0.6055 - val_accuracy: 0.6729
Epoch 5/64
14/14 [==============================] - 0s 9ms/step - loss: 0.1185 - accuracy: 0.9420 - val_loss: 0.5489 - val_accuracy: 0.8411
Epoch 6/64
14/14 [==============================] - 0s 10ms/step - loss: 0.0895 - accuracy: 0.9672 - val_loss: 0.5241 - val_accuracy: 0.8037
Epoch 7/64
14/14 [==============================] - 0s 9ms/step - loss: 0.0639 - accuracy: 0.9821 - val_loss: 0.4649 - val_accuracy: 0.8972
Epoch 8/64
14/14 [

In [27]:
from keras.models import save_model
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch)
hypermodel.save('best_model.h5')

Epoch 1/26
17/17 [==============================] - 2s 5ms/step - loss: 0.4523 - accuracy: 0.7984
Epoch 2/26
17/17 [==============================] - 0s 5ms/step - loss: 0.1531 - accuracy: 0.9583
Epoch 3/26
17/17 [==============================] - 0s 6ms/step - loss: 0.1601 - accuracy: 0.9596
Epoch 4/26
17/17 [==============================] - 0s 6ms/step - loss: 0.0918 - accuracy: 0.9708
Epoch 5/26
17/17 [==============================] - 0s 5ms/step - loss: 0.0822 - accuracy: 0.9746
Epoch 6/26
17/17 [==============================] - 0s 5ms/step - loss: 0.0934 - accuracy: 0.9715
Epoch 7/26
17/17 [==============================] - 0s 6ms/step - loss: 0.0921 - accuracy: 0.9710
Epoch 8/26
17/17 [==============================] - 0s 6ms/step - loss: 0.0876 - accuracy: 0.9581
Epoch 9/26
17/17 [==============================] - 0s 5ms/step - loss: 0.0604 - accuracy: 0.9828
Epoch 10/26
17/17 [==============================] - 0s 8ms/step - loss: 0.0798 - accuracy: 0.9714
Epoch 11/26
17/17 [

In [28]:
eval_result = hypermodel.evaluate(X_train, y_train)
print("[test loss, test accuracy]:", eval_result)

17/17 [==============================] - 0s 2ms/step - loss: 0.0405 - accuracy: 0.9850
[test loss, test accuracy]: [0.040476955473423004, 0.9850467443466187]


In [29]:
# Load the model and predict
from keras.models import load_model
hypermodel = load_model('best_model.h5')
y_pred = hypermodel.predict(X_test)
y_pred = y_pred.astype(int)

In [30]:
y_pred = pd.DataFrame(y_pred, columns=['Survived'], index=X_test.index)
# saving the dataframe 
y_pred.to_csv('Predictions.csv') 

In [31]:
import winsound
duration = 500  # milliseconds
freq = 3000  # Hz
winsound.Beep(freq, duration)